In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import datetime as dt
import time
import concurrent.futures

from typing import Dict
from session import VevenSession


async def mass_get_cookies(credentials: Dict[str, str], event_url):
    sessions = {}
    for un, pw in credentials.items():
        print('\nUser:', un)
        Session = VevenSession(
            event_url=event_url,
            username=un, password=pw
        )
        await Session.get_cookies()
        sessions[un] = Session
    return sessions


def timed_mass_register(trigger, sessions):
    while dt.datetime.now() < trigger:
        time.sleep(0.001)
    
    responses = {}
    print('\nStarting registers. Time:', dt.datetime.now())
    for un, Session in sessions.items():
        t0 = time.time()
        res = Session.register()
        if res.status_code == 201:
            print('{} registered at appx. {}\nTime elapsed: {}s'.format(
                un, dt.datetime.now(), (time.time() - t0)))
        else:
            res = Session.register()
            print('{} failed to register. Retry code: {}\nTime elapsed: {}s'.format(
                un, res.status_code, (time.time() - t0)))
        responses[Session.username] = res
        
    return responses


def threaded_timed_mass_register(trigger, session_list):
    while dt.datetime.now() < trigger:
        time.sleep(0.001)
    
    print('\nStarting registers. Time:', dt.datetime.now())
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        responses = list(executor.map(
            lambda session: session.register(), session_list
        ))

    return responses

ModuleNotFoundError: No module named 'pyppeteer'

In [ ]:
trigger = dt.datetime(year=2020, month=11, day=2, hour=20, minute=18)

event_url = 'https://omega.ntnu.no/events/show/541'
creds = {'markskag': 'Passord123'}

# Register all users in credentials to event
sessions = await mass_get_cookies(creds, event_url)

responses = threaded_timed_mass_register(trigger, sessions)

In [ ]:
trigger = dt.datetime(year=2020, month=9, day=24, hour=12)

event_url = 'https://omega.ntnu.no/events/show/541'
creds = {'markskag': 'Passord123'}

# Register all users in credentials to event
sessions = await mass_get_cookies(creds, event_url)

responses = timed_mass_register(trigger, sessions)

# Retry if some of the requests failed
failed_req_users = [user for user, res in responses.items() if res.status_code != 201]
if len(failed_req_users):
    print('Retrying for users:', failed_req_users)
    failed_sessions = {un: Session for un, Session in sessions.items() if un in failed_req_users}
    timed_mass_register(trigger, failed_sessions)

In [ ]:
# omega.ntnu.no ip: 129.241.205.135
# ping og trekk fra appx register time.

In [7]:
import pandas as pd



In [8]:
trigger, event_url, creds, attempts

(Timestamp('2020-10-02 21:00:00'), 'https://omega.ntnu.no/events/show/541', <Box: {'markskag': 'Passord123'}>, 10)

In [11]:
dict(creds)

{'markskag': 'Passord123'}